# Purpose

- Produce plots of P(DM), P(z), P(z,DM) and P(E)

In [7]:
from zdm import survey
from zdm import cosmology as cos
from zdm import loading as loading
from zdm.misc_functions import *
import zdm.iteration as it
from zdm.MCMC import calc_log_posterior
from astropy.cosmology import Planck18
import scipy.stats as st
import zdm.energetics as en

plt.rcParams['font.size'] = 16
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=["k", "tab:orange", "tab:blue", "tab:green", "tab:purple", "tab:brown", "tab:pink", "tab:gray"]) 

# Load parameter vals

- These parameter vals are taken as output from visualise_mcmc.ipynb
- Specify a dictionary of each parameter and its value, if the parameter is not specified the default value will be used

In [8]:
best_fits = []
labels = []

labels.append('Default') 
# BASE
best_fits.append({'sfr_n': 0.8808527057055584, 'alpha': 0.7895161131856694, 'lmean': 2.1198711983468064, 'lsigma': 0.44944780033763343, 'lEmax': 41.18671139482926, 'lEmin': 39.81049090314043, 'gamma': -1.1558450520609953, 'H0': 54.6887137195215})

# labels.append(r'No $P(N)$') 
# # BASE_nPn
# best_fits.append({'sfr_n': 0.8808527057055584, 'alpha': 0.7895161131856694, 'lmean': 2.075050719100272, 'lsigma': 0.40222616901185415, 'lEmax': 41.17952332700598, 'lEmin': 39.580394950607136, 'gamma': -1.055765846696212, 'H0': 51.59460417151428})

# labels.append('Power law luminosity') 
# # BASE_lf0_2
# best_fits.append({'luminosity_function': 0, 'sfr_n': 0.8778484759181941, 'alpha': 0.8651834188953196, 'lmean': 2.1268748683068246, 'lsigma': 0.3544020735989375, 'lEmax': 41.955851229232934, 'lEmin': 39.80130821833704, 'gamma': -1.6765273303795012, 'H0': 54.85781191943868})

# labels.append(r'Spectral index $\alpha$') 
# # BASE_a0
# best_fits.append({'alpha_method': 0, 'sfr_n': 0.8272410635282986, 'alpha': 0.6211735019300136, 'lmean': 2.005995812493475, 'lsigma': 0.4063898336832561, 'lEmax': 41.204693066954945, 'lEmin': 39.794705592292544, 'gamma': -1.25, 'H0': 49.28436512805642})

# labels.append(r'Source evolution') 
# # BASE_se1
# best_fits.append({'source_evolution': 1, 'sfr_n': 0.22600426164470444, 'alpha': 0.32624572355241066, 'lmean': 2.2004822865581257, 'lsigma': 0.35655443596997616, 'lEmax': 41.03952420192775, 'lEmin': 39.46394967814656, 'gamma': -0.6777402671591841, 'H0': 52.242767429628756})

# # labels.append(r'Source evolution 2') 
# # # BASE_se1
# # best_fits.append({'source_evolution': 1, 'sfr_n': 0.8808527057055584, 'alpha': 0.7895161131856694, 'lmean': 2.1198711983468064, 'lsigma': 0.44944780033763343, 'lEmax': 41.18671139482926, 'lEmin': 39.81049090314043, 'gamma': -1.1558450520609953, 'H0': 54.6887137195215})


# Load surveys and grids

In [ ]:
s_names = [
    # "FAST2",
    # "FAST2_old"
    "DSA",
    # "FAST",
    # "CRAFT_class_I_and_II", 
    # "CRAFT_ICS_892", 
    # "CRAFT_ICS_1300", 
    # "CRAFT_ICS_1632", 
    # "parkes_mb_class_I_and_II"
]

states = []
survey_list = []
grid_list = []

for best_fit in best_fits:
    state = parameters.State()
    state.set_astropy_cosmo(Planck18) 
    state.update_params(best_fit)
    # state.update_param('luminosity_function', 2)
    # state.update_param('alpha_method', 0)
    # state.update_param('sfr_n', 1.36)
    # state.update_param('alpha', 1.5)
    # state.update_param('lmean', 1.97)
    # state.update_param('lsigma', 0.92)
    # state.update_param('lEmax', 43.0)
    # state.update_param('lEmin', 30.0)
    # state.update_param('gamma', -0.63)
    # state.update_param('H0', 70.0)
    # state.update_param('DMhalo', 50.0)

    states.append(state)

    if len(s_names) != 0:
        surveys, grids = loading.surveys_and_grids(survey_names = s_names, init_state=state, repeaters=False, nz=500, ndm=1400)
    else:
        surveys = []
        grids = []

    survey_list.append(surveys)
    grid_list.append(grids)

# Get expected and observed number of FRBs and likelihoods

In [ ]:
for i in range(len(survey_list)):
    newC, llc = it.minimise_const_only(None, grid_list[i], survey_list[i])
    llsum = 0

    for j in range(len(survey_list[i])):
        s = survey_list[i][j]
        g = grid_list[i][j]

        g.state.FRBdemo.lC = newC

        llsum += it.get_log_likelihood(g,s,Pn=True, psnr=True)
    
        expected=it.CalculateIntegral(g.rates,s)
        expected *= 10**g.state.FRBdemo.lC
        observed=s.NORM_FRB

        print(s.name + " - expected, observed: " + str(expected) + ", " + str(observed))

    print(labels[i], llsum)

# Produce P(DM) and P(z)

- For each survey
- Each survey contains results from ALL parameter sets specified
- Prints P(z > zlims) and P(DM > dmlims)

In [ ]:
zlims = []
dmlims = [] #survey_list[0][0].max_dm] #, survey_list[0][1].max_dm, survey_list[0][2].max_dm]
print(dmlims)

llsum = np.zeros(len(survey_list))

# fig0 = plt.figure(figsize=[8,6])
# ax0 = fig0.add_subplot()

# fig1 = plt.figure(figsize=[8,6])
# ax1 = fig1.add_subplot()

for i in range(len(survey_list[0])):
    # fig, axs = plt.subplots(1,2, figsize=[12,4])
    # axs[0].set_title(survey_list[0][i].name)
    # axs[1].set_title(survey_list[0][i].name)

    fig0 = plt.figure(figsize=[6,4])
    ax0 = fig0.add_subplot()

    fig1 = plt.figure(figsize=[6,4])
    ax1 = fig1.add_subplot()
    pzmax = 0
    pdmmax = 0

    for j in range(len(survey_list)):
        s = survey_list[j][i]
        g = grid_list[j][i]

        # Calc pdm,pz
        rates=g.rates

        dmvals=g.dmvals
        ddm = dmvals[1] - dmvals[0]
        pdm=np.sum(rates,axis=0)

        zvals=g.zvals
        dz = zvals[1] - zvals[0]
        pz=np.sum(rates,axis=1)

        pdm = pdm/np.sum(pdm)/ddm
        pz = pz/np.sum(pz)/dz

        if np.max(pdm) > pdmmax:
            pdmmax = np.max(pdm)
        if np.max(pz) > pzmax:
            pzmax = np.max(pz)

        for zlim in zlims:
            print("P(z>" + str(zlim) + ") for survey", s.name, labels[j], np.sum(pz[zvals>zlim])*dz)

        for dmlim in dmlims:
            print("P(dm>" + str(dmlim) + ") for survey", s.name, labels[j], np.sum(pdm[dmvals>dmlim])*ddm)
        
        pobs = it.get_log_likelihood(g,s,Pn=False, psnr=False)
        pobs2 = it.get_log_likelihood(g,s,Pn=False, psnr=True)
        # print(s.name, labels[j], pobs, pobs2-pobs)
        llsum[j] += pobs2 - pobs

        # Plot
        ax0.set_xlabel(r"DM$_{\mathrm{EG}}$ (pc cm$^{-3}$)")
        ax0.set_ylabel(r"$P$(DM$_{\mathrm{EG}}$)")
        ax0.plot(dmvals, pdm, label=labels[j], zorder=100-j, linewidth=2.0)
        # ax0.plot(dmvals, pdm, label=survey_list[j][i].name, zorder=10)

        ax1.set_xlabel(r"$z$")
        ax1.set_ylabel(r"$P(z)$")
        ax1.plot(zvals, pz, label=labels[j], zorder=100-j, linewidth=2.0)
        # ax1.plot(zvals, pz, label=survey_list[j][i].name, zorder=10)

    ax0.vlines(s.DMEGs, np.zeros(len(s.DMs)), pdmmax*np.ones(len(s.DMs)), ls='--', colors='r', alpha=0.7)
    ax1.vlines(s.Zs, np.zeros(len(s.Zs)), pzmax*np.ones(len(s.Zs)), ls='--', colors='r', alpha=0.7)
    ax1.vlines(s.ignored_Zs, np.zeros(len(s.ignored_Zs)), pzmax*np.ones(len(s.ignored_Zs)), ls='--', colors='c', alpha=1.0)

    ax0.set_xlim(xmax=2000, xmin=-100)
    ax0.set_ylim(ymin=0.0)
    ax0.ticklabel_format(axis='y', style='sci', scilimits=(-3,-3))

    ax1.set_xlim(xmax=1.5, xmin=0.0)
    ax1.set_ylim(ymin=0.0)

    l0 = ax0.legend(loc='upper right')
    l0.set_zorder(101)
    l0.get_frame().set_alpha(None)
    l1 = ax1.legend()
    l1.set_zorder(101)
    l1.get_frame().set_alpha(None)

    fig0.tight_layout()
    fig1.tight_layout()

    # fig0.savefig(s.name + "pdm.pdf")
    # fig1.savefig(s.name + "pz.pdf")

# print(labels)
# print(llsum)

# P(z,DM)

In [ ]:
# Survey indexes
for j in [0,1,2]:
    print(survey_list[0][j].name)

    # Parameter set indexes
    for i in [0]: #range(len(grid_list)):
        
        g = grid_list[i][j]
        
        if survey_list[i][j].zlist is None:
            Zs = None
            DMs = None
        else:
            Zs = survey_list[i][j].Zs[survey_list[i][j].zlist]
            DMs = survey_list[i][j].DMEGs[survey_list[i][j].zlist]

        # Redshifts that are ignored
        if len(survey_list[i][j].ignored_Zs) == 0: # is None:
            ignored_Zs = None
            ignored_DMs = None

            if survey_list[i][j].nozlist is not None:
                DMlines = survey_list[i][j].DMEGs[survey_list[i][j].nozlist]
            else:
                DMlines = None
        else:
            ignored_Zs = survey_list[i][j].ignored_Zs
            ignored_DMs = survey_list[i][j].DMEGs[survey_list[i][j].ignored_Zlist]

            line_list = [k for k in survey_list[i][j].nozlist if k not in survey_list[i][j].ignored_Zlist]
            DMlines = survey_list[i][j].DMEGs[line_list]

            print(survey_list[i][j].nozlist)
            print(survey_list[i][j].ignored_Zlist)

        ax = plot_grid_2(
                zDMgrid=g.rates,
                zvals=g.zvals,
                dmvals=g.dmvals,
                zmax=2.5,
                DMmax=3000,
                norm=0,
                log=True,
                project=False,
                Aconts=[0.01,0.1,0.5],
                showplot=True,
                save=False,
                name=survey_list[0][j].name + "pzdm.pdf",
                DMlines=DMlines,
                DMlims=[survey_list[0][j].max_dm],
                FRBZ=Zs,
                FRBDM=DMs,
                FRBZ2=ignored_Zs,
                FRBDM2=ignored_DMs,
                data_clr='r',
                data_clr2='tab:blue'
            )

In [ ]:
# # Parameter set indexes
# for i in [0]: #range(len(grid_list)):
    
#     Zs = []
#     DMs = []
#     DMlines = []

#     # Survey indexes
#     js = [0,1,2]
#     rates = np.zeros(grid_list[i][js[0]].rates.shape)
#     for j in js:
#         print(survey_list[i][j].name)
        
#         g = grid_list[i][j]
        
#         if survey_list[i][j].zlist is not None:
#             Zs = Zs + list(survey_list[i][j].Zs[survey_list[i][j].zlist])
#             DMs = DMs + list(survey_list[i][j].DMEGs[survey_list[i][j].zlist])

#         if survey_list[i][j].nozlist is not None:
#             DMlines = DMlines + list(survey_list[i][j].DMEGs[survey_list[i][j].nozlist])

#         rates += g.rates

#     Zs = np.array(Zs)
#     DMs = np.array(DMs)
#     DMlines = np.array(DMlines)
#     rates /= len(js)

#     if len(Zs) == 0:
#         Zs = None
#         DMs = None

#     ax = plot_grid_2(
#             zDMgrid=rates,
#             zvals=g.zvals,
#             dmvals=g.dmvals,
#             zmax=2.5,
#             DMmax=3000,
#             norm=0,
#             log=True,
#             project=False,
#             conts=False,
#             showplot=True,
#             save=True,
#             name="CRAFTpzdm.pdf",
#             DMlines=DMlines,
#             DMlims=None,
#             FRBZ=Zs,
#             FRBDM=DMs,
#             data_clr='r',
#             data_clr2='tab:blue'
#         )

# P(E)

In [ ]:
import emcee

In [ ]:
reader = emcee.backends.HDFBackend('../../zdm/mcmc/BASE_2' + '.h5')
sample = reader.get_chain(discard=200, thin=30, flat=True)

In [ ]:
for j in range(len(survey_list)):
    
    fig = plt.figure()
    lEmax=best_fits[j]['lEmax']
    lEmin=best_fits[j]['lEmin']
    gamma=best_fits[j]['gamma']

    lEvec = np.linspace(38, 46, 100)
    pE = en.vector_diff_gamma(10**lEvec, 10**lEmin, 10**lEmax, gamma)

    lEvec = lEvec[pE > 0]
    pE = pE[pE > 0]
    # pE = 1 - pE

    plt.xlabel(r'log$\,E$ (erg)')
    plt.ylabel(r'log$\,P(E)$')
    plt.ylim(bottom=-60, top=-38)
    plt.xlim(left=38, right=44)
    # plt.vlines([best_fits[j]['lEmin']],[np.log10(np.min(pE))],[np.log10(np.max(pE))], linewidth=2.0, colors='k', zorder=100, linestyle='-.')
    # plt.vlines([lEmax],[np.log10(np.min(pE))],[np.log10(np.max(pE))], linewidth=2.0, colors='k', zorder=100, linestyle='-.')
    plt.vlines([best_fits[j]['lEmin']],[-70],[-30], linewidth=3.0, colors='k', zorder=100, linestyle='-.')
    plt.vlines([lEmax],[-70],[-30], linewidth=3.0, colors='k', zorder=100, linestyle='-.')
    plt.plot(lEvec, np.log10(pE), 'k', linewidth=3.0, zorder=50)

    print(sample.shape)
    for k in range(sample.shape[0]):
        lEmax=sample[k][4]
        lEmin=sample[k][5]
        gamma=sample[k][6]

        lEvec = np.linspace(38, 46, 100)
        pE = en.vector_diff_gamma(10**lEvec, 10**lEmin, 10**lEmax, gamma)

        lEvec = lEvec[pE > 0]
        pE = pE[pE > 0]
        # pE = 1- pE
        plt.plot(lEvec, np.log10(pE), 'tab:gray', linewidth=2.0, alpha=0.05)

    lElines = np.array([])
    colors = ['r','c','m','y','o','k']
    for i in range(len(survey_list[0])):
        survey = survey_list[j][i]
        grid = grid_list[j][i]

        dmvals = grid.dmvals
        zvals = grid.zvals
        # print(survey.beam_b)
        # print(survey.beam_o)

        zlists = [survey.zlist, survey.ignored_Zlist]
        Zobss = [survey.Zs[survey.zlist], survey.ignored_Zs]
        colours = ['r','c']
        
        for k, zlist in enumerate(zlists):
            if zlist is not None:
                Zobs = Zobss[k]
                # Evaluate thresholds at the exact DMobs
                ddm = dmvals[1] - dmvals[0]
                kdmobs=(survey.DMs - survey.DMhalo - survey.meta['DMG'])/ddm
                kdmobs=kdmobs[zlist]
                kdmobs[kdmobs<0] = 0
                idmobs1=kdmobs.astype('int')
                idmobs2=idmobs1+1
                dkdmobs=kdmobs-idmobs1 # applies to idms2

                # get indices in z space
                dz=zvals[1]-zvals[0]
                kzs=Zobs/dz
                izs1=kzs.astype('int')
                izs2=izs1+1
                dkzs=kzs-izs1 # applies to izs2

                # Linear interpolation
                Eths = grid.thresholds[:,izs1,idmobs1]*(1.-dkdmobs)*(1-dkzs)
                Eths += grid.thresholds[:,izs2,idmobs1]*(1.-dkdmobs)*dkzs
                Eths += grid.thresholds[:,izs1,idmobs2]*dkdmobs*(1-dkzs)
                Eths += grid.thresholds[:,izs2,idmobs2]*dkdmobs*dkzs

                # bEobs = np.zeros(Eths.shape)
                # for k,b in enumerate(survey.beam_b):
                #     bEths = Eths / b
                #     bEobs += bEths*survey.Ss[zlist]
                
                bav = np.sum(survey.beam_b * survey.beam_o) / np.sum(survey.beam_o)  #sum(b P(b))
                bEobs = Eths*survey.Ss[zlist] / bav
                bEobs = bEobs[0]

                print(survey.frbs["TNS"].values[zlist])
                print(np.log10(bEobs))

                # lElines = np.concatenate((lElines, np.log10(bEobs)))
                # plt.hist(np.log10(bEobs), bins=lEvec, density=True, cumulative=True, histtype='step', color=colors[i], label=survey.name)
                
                # plt.vlines(np.log10(bEobs), np.ones(len(bEobs)) * np.log10(np.min(pE)), np.ones(len(bEobs)) * np.log10(np.max(pE)), ls='--', colors=colours[k])
                plt.vlines(np.log10(bEobs), np.ones(len(bEobs)) * -70, np.ones(len(bEobs)) * -30, ls='--', colors=colours[k])

    # plt.hist(lElines, bins=lEvec, density=True, cumulative=True, histtype='step')
    # plt.legend()
    plt.tight_layout()
    plt.savefig("pE.pdf")
        